In [7]:
import pandas as pd
from groq import Groq
import itertools
import tqdm
import time
import os
import random
import re

In [8]:
data = []
for file in list(filter(lambda x: x.startswith('p-'), os.listdir('./md_files/'))):
    with open('./md_files/' + file, 'r', encoding='utf-8') as file:
        content = file.read()
        chunks = list(map(lambda x: re.sub(r"# ACÓRDÃO.*\n", "", x).strip(), filter(lambda x: x not in ['RELATÓRIO', 'VOTO'], re.split(r"# (RELATÓRIO|VOTO)", content))))
        data.append({
            "acordao": chunks[0],
            "relatorio": chunks[1],
            "voto": chunks[2]
        })
df = pd.DataFrame(data)
df

,acordao,relatorio,voto
0,"Vistos, relatados e discutidos os autos do pro...",1. Trata o presente processo da Prestação de C...,"Em face dos dados apresentados nos autos, veri..."
1,"Vistos, relatados e discutidos os autos do pro...",Tratam os autos da aposentadoria do servidor p...,Trata-se de apreciação da legalidade e registr...
2,"Vistos, relatados e discutidos os autos do pro...",1. Tratam os autos de Denúncia protocolada na ...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...
3,"Vistos, relatados e discutidos os autos do pro...",1. Trata-se de apreciação da legalidade e regi...,A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...
4,"Vistos, relatados e discutidos os autos do pro...",Tratam os autos da aposentadoria da servidora ...,Trata-se de apreciação da legalidade e registr...
...,...,...,...
145,"Vistos, relatados e discutidos os autos do pro...","1. Trata-se de INSPEÇÃO, com o objetivo de ver...",A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...
146,"Vistos, relatados e discutidos os autos do pro...",Trata o presente processo de aposentadoria vol...,O EXMO. SENHOR CONSELHEIRO VALMIR GOMES RIBEIR...
147,"Vistos, relatados e discutidos os autos do pro...",Tratam os autos da aposentadoria da servidora ...,Trata-se de apreciação da legalidade e registr...
148,"Vistos, relatados e discutidos os autos do pro...",1. Trata-se de apreciação da legalidade e regi...,A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...


### Examples Definition

In [9]:
example1 = """
Relatório:
Trata o presente processo de aposentadoria voluntária por tempo de contribuição da servidora pública estadual Eutália Paz da Silva, no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal do Estado da Secretaria de Estado de Saúde, pelas regras previstas no artigo 3º da Emenda Constitucional nº 47, de 05 de julho de 2005.

Da análise dos autos e de acordo com o Relatório Técnico (fls. 94-95), apurou-se que:

1. A servidora conta com a idade de 57 (cinquenta e sete) anos e 37 (trinta e sete) anos e 195 (cento e noventa e cinco) dias de contribuição, para fins de aposentadoria;
2. A aposentadoria foi concedida no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal da Secretaria de Estado de Saúde, através da Portaria nº 98 de 10/02/2021;
3. Os proventos mensais iniciais foram calculados no valor de R$1.942,42 (mil novecentos e quarenta e dois reais e quarenta e dois centavos), conforme Ato de Fixação dos Proventos (fls. 79);
4. Atualmente a servidora faz jus aos proventos no valor de R$ 2.341,74 (dois mil trezentos e quarenta e um reais e setenta e quatro centavos), de acordo com a Ficha Financeira atualizada (fls. 96-157).

O Ministério Público de Contas se manifestou (fls. 162-163) em pronunciamento da lavra da Procuradora Anna Helena de Azevedo Lima, tendo se manifestado favorável pela legalidade e registro da matéria.

É o RELATÓRIO.

Voto:
O EXMO. SENHOR CONSELHEIRO VALMIR GOMES RIBEIRO (Relator):

Visto e analisado o presente processo, e ainda consubstanciado na Análise Técnica (fls. 94-95), bem como no Parecer do Ministério Público de Contas (fls. 162-163), concluo votando:

1. pela legalidade e registro da aposentadoria voluntária por tempo de contribuição da servidora pública estadual Eutália Paz da Silva, CPF nº 112.709.342-87, matrícula nº 91456-1, no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal do Estado da Secretaria de Estado de Saúde, concedida através da Portaria nº 98 de 10/02/2021, com fundamento no artigo 3º da Emenda Constitucional nº 47, de 05 de julho de 2005;
2. pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada. Se impresso, para conferência acesse o site http://www.tceac.tc.br/conferencia e informe o código 01335634.
3. após, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, na 127ª Sessão Ordinária Virtual, ACORDAM os Membros da 2ª Câmara do Tribunal de Contas do Estado do Acre, à unanimidade, nos termos do voto do Conselheiro-Relator, com fulcro no artigo 61, inciso III, da Constituição Estadual:

1. pela legalidade e registro da aposentadoria voluntária por tempo de contribuição da servidora pública estadual Eutália Paz da Silva, CPF nº 112.709.342-87, matrícula nº 91456-1, com fundamento no artigo 3º da Emenda Constitucional nº 47, de 05 de julho de 2005, no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal do Estado da Secretaria de Estado de Saúde, concedida através da Portaria nº 98 de 10/02/2021;
2. pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

In [10]:
example2 = """
Relatório:
Trata o presente processo de aposentadoria voluntária integral especial por tempo de contribuição da servidora pública estadual Marizete Correia de Andrade Carneiro, no cargo de Professora P2, 30 horas, Classe II, Referência J, do Quadro de Pessoal do Estado da Secretaria de Estado de Educação, Cultura e Esportes, pelas regras previstas no artigo 5º, §1º, da Emenda Constitucional do Estado nº 52, de 02 de dezembro de 2019 e §2º, inciso I, do mesmo artigo.

Da análise dos autos e de acordo com o Relatório Técnico (fls. 162-164), apurou-se que:

1. A servidora conta com a idade de 52 (cinquenta e dois) anos e 34 (trinta e quatro) anos e 154 (cento e cinquenta e quatro) dias de contribuição, para fins de aposentadoria;
2. A aposentadoria foi concedida no cargo de Professora P2, 30 horas, Classe II, Referência 6, do Quadro de Pessoal da Secretaria de Estado de Educação, Cultura e Esportes, através da Portaria nº 613 de 20/09/2022;
3. Os proventos mensais iniciais foram calculados no valor de R$ 5.148,03 (cinco mil cento e quarenta e oito reais e três centavos), conforme Ato de Fixação dos Proventos (fl. 138);
4. Observa-se, no entanto, que o enquadramento da servidora no ato de aposentadoria foi equivocado, uma vez que, observando-se o art. 29 da LCE nº 67/99 com redação dada pelo art. 29, §8º, da LCE nº 330/17, a servidora deveria ter sido aposentada na Referência “J”, e seus proventos mensais, com a devida correção, ter sido fixado em R$ 6.215,78 (seis mil duzentos e quinze reais e setenta e oito centavos);
5. atualmente a servidora faz jus aos proventos no valor de R$ 7.017,77 (sete mil e dezessete reais e setenta e sete centavos), de acordo com a Ficha Financeira atualizada (fls. 165-216).

O Ministério Público de Contas se manifestou (fls. 221) em pronunciamento da lavra do Procurador Anna Helena de Azevedo Lima, tendo se manifestado favorável pela legalidade e registro da matéria, sem prejuízo da cientificação da beneficiária a respeito e assinação de prazo para a correção da origem.

É o RELATÓRIO.

Voto:
O EXMO. SENHOR CONSELHEIRO VALMIR GOMES RIBEIRO (Relator):

Visto e analisado o presente processo, e ainda consubstanciado na Análise Técnica (fls. 162-164), bem como no Parecer do Ministério Público de Contas (fls. 221), concluo votando:

1. pela legalidade e registro da aposentadoria voluntária integral especial por tempo de contribuição da servidora pública estadual Marizete Correia de Andrade Carneiro, CPF nº 308.350.892-15, matrícula nº 210366-1, no cargo de Professora P2, 30 horas, Classe II, Referência J, do Quadro de Pessoal do Estado da Secretaria de Estado de Educação, Cultura e Esportes, concedida através da Portaria nº 613 de 20/09/2022, com fundamento no artigo 5º, §1º, da Emenda Constitucional do Estado nº 52, de 02 de dezembro de 2019 e §2º, inciso I, do mesmo artigo;
2. pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada.
3. após, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, na 127ª Sessão Ordinária Virtual, ACORDAM os Membros da 2ª Câmara do Tribunal de Contas do Estado do Acre, à unanimidade, nos termos do voto do Conselheiro-Relator, com fulcro no artigo 61, inciso III, da Constituição Estadual: 1) pela legalidade e registro da aposentadoria voluntária integral especial por tempo de contribuição da servidora pública estadual Marizete Correia de Andrade Carneiro, CPF nº 308.350.892-15, matrícula nº 210366-1, com fundamento no artigo 5º, §1º, da Emenda Constitucional do Estado nº 52, de 02 de dezembro de 2019 e §2º, inciso I, do mesmo artigo, no cargo de Professora P2, 30 horas, Classe II, Referência J, do Quadro de Pessoal do Estado da Secretaria de Estado de Educação, Cultura e Esportes, concedida através da Portaria nº 613 de 20/09/2022; 2) pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

In [11]:
example3 = """
Relatório:
1. Tratam os autos da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula nº 266027-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J.
2. A servidora foi admitida na Secretaria Estadual de Educação por concurso público em 1º de dezembro de 1993 para ocupar o cargo de Professora PE5, 40 horas. Conforme fichas financeiras da servidora esta recebeu todas as promoções e progressões da carreira do magistério, nos termos da Lei complementar n. 67/1999 e, nos termos desta Lei foi enquadrada como professora P2, 25 horas, a partir de 16 de junho 1999. Posteriormente, por meio da LC n. 91, de 14.02.2001 que alterou a LC n. 67/99 a servidora foi enquadrada como Professora Nível Superior P2, 30 horas a partir de 1º de fevereiro de 2001. Após a servidora comprovar a formação superior para o exercício do cargo.
3. A servidora requereu a aposentadoria em 27 de novembro de 2018, fl. 42, pelo exercício do magistério na Escola Dom Julio Mattioli, no município de Sena Madureira, tendo sido efetuada a concessão de aposentadoria através da Portaria nº 79 de 26/01/2023 do Instituto de Previdência do Estado do Acre, publicada no DOE nº 13.464, de 30/01/2023;
4. O tempo de serviço líquido, apurado para fins da aposentadoria especial do magistério foi de 30 anos e 259 dias de contribuição, atendendo os requisitos para aposentadoria na modalidade voluntária integral especial por tempo de contribuição, nos termos do art. 4º da Emenda Constitucional nº 52, de 2 de dezembro de 2019, com proventos integrais de acordo com o § 6º, inciso I, do mesmo artigo.
5. A 4º IGCE deste Tribunal afirma, às fls. 203/204, que a servidora preenche todos os requisitos constitucionais para fazer jus à aposentadoria com enquadramento da fixação dos proventos em P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria estadual de Educação, com benefício iniciado em 30/01/2023, no valor de R$ 6.215,78 (seis mil duzentos e quinze reais e setenta e oito centavos)¹, conforme ao enquadramento final da carreira, mencionado no Ato de Fixação dos Proventos à fl. 21;
6. O Ministério Público de Contas pronunciou-se à fl. 262, através da sua Ilustre Procuradora, Dra. Anna Helena de Azevedo Lima.

É o Relatório.

Voto:
A EXMA. SENHORA CONSELHEIRA NALUH MARIA LIMA GOUVEIA (Relatora):

1. Trata-se de apreciação da legalidade e registro da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula nº 266027-1, no cargo de Professora de Nível Superior, 30 horas, Classe II, Referência J.
2. Posto isso, e tudo o que mais consta dos autos, e considerando a regularidade da documentação apresentada, assim como sua conformidade com a legislação vigente pertinente, inclusive levando em conta as análises do corpo técnico e do Ministério Público de Contas, e com fundamento no art. 61, inciso III, da Constituição Estadual de 1989, VOTO:

1) Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula n. 229962-1, no cargo de Professora, P2-30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 79, de 26 de janeiro de 2023, na modalidade aposentadoria voluntária Especial por tempo de contribuição, nos termos art. 4º, da Emenda Constitucional nº 52, de 2 de dezembro de 2019, com proventos integrais, de acordo com o § 6º, inciso I, do mesmo artigo, especificados no Ato de Fixação à fl. 21;
2) Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão, e
3) Após as formalidades de estilo, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, acordam os Membros da 1ª Câmara do Tribunal de Contas do Estado do Acre na 119ª Sessão Ordinária Virtual, por unanimidade, nos termos do voto da Conselheira-Relatora, em:

1. Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula n. 266027-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 79, de 26 de janeiro de 2023, na modalidade aposentadoria voluntária Especial por tempo de contribuição, nos termos art. 4º, da Emenda Constitucional nº 52, de 2 de dezembro de 2019, com proventos integrais, de acordo com o § 6º, inciso I, do mesmo artigo, especificados no Ato de Fixação à fl. 21;
2. Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre- ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

In [12]:
example4 = """
Relatório:
1. Trata-se de apreciação da legalidade e registro da aposentadoria voluntária por tempo de contribuição da SR.ª LUCÉLIA NAGÃO GONÇALVES, matriculada na Secretaria de Estado de Educação e Esporte sob o número 9130438-3, com proventos calculados pela média1, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, consoante se vê pela Portaria n. 863, de 07 de julho de 2017 (DOE n. 12.092, de 11-07-2017) - fls. 72/73, apresentada nesta Corte em 13-08-20192.

2. Quanto aos proventos, foram fixados em R$ 3.423,40 (três mil quatrocentos e vinte e três reais e quarenta centavos), no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, nos termos do Ato de Fixação dos Proventos à fl. 683, de acordo com o previsto no Anexo VII, da Lei Complementar Estadual n. 67, de 29-06-1999, modificado pela LCE n. 330, de 06 de março de 20174.

3. Após o registro, a autuação e a distribuição, os autos foram enviados à DIRETORIA DE AUDITORIA FINANCEIRA E ORÇAMENTÁRIA, que se manifestou por meio da 4ª INSPETORIA GERAL DE CONTROLE EXTERNO, pela admissibilidade da concessão de aposentadoria, em razão do preenchimento dos requisitos legais pertinentes à matéria - fls. 80/83.

4. O MINISTÉRIO PÚBLICO DE CONTAS, por meio de seu i. Procurador, Dr. João Izidro de Melo Neto, pronunciou-se à fl. 120.

5. É o breve Relatório.

Voto:
A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO DE ARAÚJO (Relatora):

1. Trata-se de apreciação da legalidade e registro da concessão de aposentadoria à SR.ª LUCÉLIA NAGÃO GONÇALVES, com proventos calculados pela média, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”.
2. Observa-se que após a edição da LCE n. 330/2017, que modificou o § 8º do artigo 29 da LCE n. 67/1999, foi determinado o reenquadramento dos servidores, observando-se o interstício de trinta e três meses para cada progressão.
3. Ante todo o exposto, e tudo o que mais consta dos autos, bem como considerando a regularidade da documentação apresentada e a conformidade do ato de concessão com o estabelecido na legislação em vigor, inclusive levando-se em conta as manifestações do corpo técnico e do Ministério Público de Contas, VOTO, com base no artigo 61, inciso III, da Constituição Estadual de 1989, pelo:
   3.1 reconhecimento da LEGALIDADE e, consequente, REGISTRO neste Tribunal de Contas, da aposentadoria voluntária por tempo de contribuição da SR.ª LUCÉLIA NAGÃO GONÇALVES, matrícula n. 9130438-3, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, do quadro de pessoal do Estado da Secretaria de Educação e Esporte, nos termos do artigo 40, § 1º, inciso III, alínea “a”, da Constituição Federal, com a redação dada pela Emenda Constitucional n° 41, de 19 de dezembro de 20035, com proventos fixados em R$ 3.423,40 (três mil quatrocentos e vinte e três reais e quarenta centavos), nos termos do Ato de Fixação dos Proventos à fl. 68, de acordo com o previsto no Anexo VII, da Lei Complementar Estadual n. 67, de 29-06-1999, modificado pela LCE n. 330, de 06 de março de 2017;
   3.2. ENCAMINHAMENTO de cópia do processo ao Instituto de Previdência do Estado do Acre, para proceder à ciência da interessada do teor da decisão, bem como para que observe o artigo 11, da Resolução-TCE n. 103, de 14-09-2016; e
   3.3. finalmente, após as formalidades de estilo, pelo ARQUIVAMENTO dos autos.

4. É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, ACORDAM os Membros da 2ª Câmara do Tribunal de Contas do Estado do Acre, na 127ª Sessão Ordinária Virtual, POR UNANIMIDADE, nos termos do voto da Conselheira-Relatora, em:

1. RECONHECER a LEGALIDADE e, consequentemente, REGISTRAR neste Tribunal de Contas, a aposentadoria voluntária por tempo de contribuição da SR.ª LUCÉLIA NAGÃO GONÇALVES, matrícula n. 9130438-3, com proventos calculados pela média, concedida por meio da Portaria n. 863, de 07 de julho de 2017, com fundamento no artigo 40, § 1º, inciso III, alínea “a”, da Constituição Federal, com a redação dada pela Emenda Constitucional n° 41, de 19 de dezembro de 2003, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, do quadro de pessoal do Estado da Secretaria de Educação e Esporte, com proventos fixados em R$ 3.423,40 (três mil quatrocentos e vinte e três reais e quarenta centavos), nos termos do Ato de Fixação dos Proventos à fl. 68, de acordo com o previsto no Anexo VII, da Lei Complementar Estadual n. 67, de 29-06-1999, modificado pela LCE n. 330, de 06 de março de 2017;
2. ENCAMINHAR cópia do processo ao Instituto de Previdência do Estado do Acre, para proceder à ciência da interessada do teor da decisão, bem como para que observe o artigo 11, da Resolução-TCE n. 103, de 14-09-2016; e c) ARQUIVAR os autos, após as formalidades de estilo.
"""

In [13]:
example5 = """
Relatório:
1. Tratam os autos da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra Arinete Oliveira Souza Barros, matrícula nº 244503-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J.
2. A servidora foi admitida na Secretaria Estadual de Educação por concurso público em 22 de maio de 1992 para ocupar o cargo de Professora PS2. Posteriormente foi enquadrada no cargo P1, 25 horas, por meio da Portaria n. 935, de 25/01/2002, recebendo todas as promoções e progressões da carreira do magistério, nos termos da Lei complementar n. 67/1999.
3. Com base na Lei Complementar n. 213, de 18/06/2010, obteve progressão funcional para o cargo de professora de Nível Superior 30 horas, a partir de outubro daquele mesmo ano. Após a servidora comprovar a formação em nível superior em pedagogia, concluindo no ano de 2010 e especialização em 2011.
4. A servidora requereu a aposentadoria em 2 de janeiro de 2018, tendo sido efetuada a concessão de aposentadoria através da Portaria nº 242 de 02/04/2018 do Instituto de Previdência do Estado do Acre, publicada no DOE nº 12.272 de 03/04/2018;
5. O tempo de serviço líquido, apurado para fins da aposentadoria especial do magistério foi de 26 anos e 221 dias de contribuição, atendendo os requisitos para aposentadoria na modalidade voluntária integral especial por tempo de contribuição, nos termos do art. 6 º, incisos I, II, III e IV, da Emenda Constitucional nº 41, de 19 de dezembro de 2003, c/c art. 2º, da Emenda Constitucional n.º 47, de 05 de julho de 2005, e art. 95 da Lei Complementar Estadual n. 154/2005;

6. A 4º IGCE deste Tribunal afirma, às fls. 62/63 que a servidora preenche todos os requisitos constitucionais para fazer jus à aposentadoria com enquadramento da fixação dos proventos em P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria estadual de Educação, com benefício iniciado em 03/04/2018, no valor de R$ 5.725,39 (cinco mil setecentos e vinte e cinco reais e trinta e nove centavos)1, conforme ao enquadramento final da carreira, mencionado no Ato de Fixação dos Proventos à fl. 59, e

7. O Ministério Público de Contas pronunciou-se à fl. 122, através do seu Ilustre Procurador, Dr. Sérgio Cunha Mendonça.

É o Relatório.

Voto:
A EXMA. SENHORA CONSELHEIRA NALUH MARIA LIMA GOUVEIA (Relatora):

1. Trata-se de apreciação da legalidade e registro da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra. Arinete Oliveira Souza Barros, matrícula nº 244503-1, no cargo de Professora de Nível Superior, 30 horas, Classe II, Referência J.
2. Posto isso, e tudo o que mais consta dos autos, e considerando a regularidade da documentação apresentada, assim como sua conformidade com a legislação vigente pertinente, inclusive levando em conta as análises do corpo técnico e do Ministério Público de Contas, e com fundamento no art. 61, inciso III, da Constituição Estadual de 1989, VOTO:

1) Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Arinete Oliveira Souza Barros, matrícula n. 244503-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 242, de 2 de abril de 2018, na modalidade aposentadoria voluntaria Especial por tempo de contribuição, nos termos do art. 6º, incisos I, II, III e IV, da Emenda Constitucional nº 41, de 19 de dezembro de 2003, c/c art. 2º, da Emenda Constitucional n.º 47, de 05 de julho de 2005, e art. 95 da Lei Complementar Estadual n. 154/2005, com proventos estabelecidos no Ato de Fixação à fl. 59;

2) Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão, e

3) Após as formalidades de estilo, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, acordam os Membros da 1ª Câmara do Tribunal de Contas do Estado do Acre na 119ª Sessão Ordinária Virtual, por unanimidade, nos termos do voto da Conselheira-Relatora, em:
1. Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Arinete Oliveira Souza Barros, matrícula n. 244503-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 242, de 2 de abril de 2018, na modalidade aposentadoria voluntária Especial por tempo de contribuição, nos termos do art. 6º, incisos I, II, III e IV, da Emenda Constitucional nº 41, de 19 de dezembro de 2003, c/c art. 2º, da Emenda Constitucional n.º 47, de 05 de julho de 2005, e art. 95 da Lei Complementar Estadual n. 154/2005, com proventos estabelecidos no Ato de Fixação à fl. 59;
2. Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre- ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

### Promps Definition

In [14]:
base_prompt = """
Você é um assistente jurídico especializado na redação de acórdãos do Tribunal de Contas do Estado do Acre. A partir das informações fornecidas, você deve elaborar um acórdão completo que siga uma estrutura clara, coesa e que inclua os seguintes elementos em um texto corrido:

1. **Cabeçalho**: Inicie o acórdão com o nome do tribunal (Tribunal de Contas do Estado do Acre), o número do processo e a data de julgamento.
2. **Relator e Membros do Órgão Julgador**: Identifique o relator do processo, bem como os demais membros do órgão colegiado (câmara, turma, seção, órgão especial, plenário etc.) que participaram do julgamento. Informe também o resultado da votação (unânime ou com voto vencido).
3. **Resumo dos Fatos**: Resuma os fatos principais do processo, incluindo as alegações, os documentos relevantes e os argumentos apresentados pelas partes envolvidas.
4. **Fundamentação**: Apresente os fundamentos jurídicos da decisão. Isso deve incluir a análise das questões de direito envolvidas, as normas aplicáveis e a interpretação dos membros do órgão colegiado.
5. **Decisão**: Expresse de forma clara e objetiva a decisão do colegiado, indicando os resultados do julgamento e as providências a serem tomadas, caso aplicável (por exemplo, aprovação ou reprovação de contas, determinação de devolução de recursos, etc.).
6. **Voto Vencido** (se houver): Caso a votação não seja unânime, transcreva o voto vencido de um dos membros, expondo sua argumentação divergente.\n\n
"""

In [15]:
zero_shot_prompt = base_prompt + """
Com base no relatório e no voto abaixo, redija o texto do acordão:

Relatório:
{relatorio}

Voto:
{voto}

Acordão:
"""

In [16]:
one_shot_prompt = base_prompt + """
Exemplo:
{example}

A partir do exemplo acima, 
""" + zero_shot_prompt

In [17]:
def generate_few_shot_prompt(examples):
    prompt = base_prompt
    for i in range(len(examples)):
        prompt += f"""
        Exemplo {i + 1}:
        {examples[i]}\n\n
        """
    return """
        A partir dos exemplos acima, 
        """ + zero_shot_prompt

### Experiment

In [18]:
keys = ['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', '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', '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', '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', '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', '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', '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', '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', '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']


In [19]:
def generate(model, prompt, temperature):
    random.shuffle(keys)
    for key in keys:
        try:
            client = Groq(api_key=key)
            response = client.chat.completions.create (
                messages=[{"role": "user", "content": prompt}],
                model=model,
                temperature=temperature,
            )
            return response.choices[0].message.content
        except:
            print("key expirada")

In [20]:
examples = [example1, example2, example3, example4, example5]
models = ["llama-3.1-8b-instant"]
temperatures = [0.25, 0.5, 0.75, 1.0]
n_exemples = [0, 1, 2, 3, 4, 5]

In [ ]:
for n_example, model, temp in itertools.product(n_exemples, models, temperatures):
    results = []
    filename = f'results/{model}-{n_example}-examples-temperature-{temp}.csv'
    if not os.path.isfile(filename):
        with tqdm.tqdm(total=len(df), smoothing=0.1) as pbar:
            for idx, row in df.iterrows():
                relatorio = row["relatorio"]
                voto = row["voto"]
                prompt = ""
                if n_example == 0:
                    prompt = zero_shot_prompt.format(
                        relatorio=relatorio,
                        voto=voto,
                    )
                elif n_example == 1:
                    prompt = one_shot_prompt.format(
                        relatorio=relatorio,
                        voto=voto,
                        example=examples[0]
                    )
                else:
                    print(n_example)
                    prompt = generate_few_shot_prompt(examples[:n_example]).format(
                        relatorio=relatorio,
                        voto=voto,
                    )

                response = ""
                while True:
                    try:
                        response = generate(model, prompt, temp)
                        break
                    except Exception as e:
                        print('Error: ' + str(e) + ' (Retrying)')
                        time.sleep(10)

                results.append({
                    "relatorio": relatorio,
                    "voto": voto,
                    "acordao": row['acordao'],
                    "generated": response,
                    "model": model,
                    'n_examples': n_example,
                    "temperature": temp
                })

                pd.DataFrame(results).to_csv(filename, index=False)
                pbar.update(1)

  7%|▋         | 11/150 [00:24<05:01,  2.17s/it]

key expirada


 41%|████      | 61/150 [02:02<02:53,  1.95s/it]

key expirada


 53%|█████▎    | 80/150 [02:38<02:11,  1.88s/it]

...

key expirada
3
key expirada
key expirada
key expirada
key expirada
key expirada
key expirada
key expirada
key expirada


 61%|██████▏   | 92/150 [05:18<03:37,  3.75s/it]

key expirada
3
key expirada
key expirada
key expirada
key expirada
key expirada
key expirada
key expirada
key expirada


 62%|██████▏   | 93/150 [05:22<03:35,  3.77s/it]

key expirada
3
key expirada
key expirada
key expirada
key expirada
